# 求解弹簧质量逆问题

神经网络求解器相较于传统数值方法的另一个重要优势是其能够求解逆问题。在逆问题中，我们从一组观测值开始，然后利用这些观测值计算产生这些观测值的因果因素。为了演示这一概念，我们将在逆环境中求解相同的弹簧质量系统。具体来说，我们假设三个质量块 $x_1, x_2 \text{ 和 } x_3$ 的位移数据可以通过解析解获得，而第一个质量块 ($m_1$) 和最后一个弹簧系数 ($k_4$) 是未知的。我们将使用神经网络从解析解中吸收位移，并用它来反演微分方程中的未知量。

## 案例设置

本部分的方程文件保持不变。在这里，我们将使用 [`PointwiseConstraint`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.domain.constraint.html#modulus.domain.constraint.continuous.PointwiseConstraint) 类来吸收训练数据中的解。我们将创建一个网络，通过在 $t$ 和 $(x_1, x_2, x_3)$ 之间建立映射来记忆这三个位移。第二个网络将被训练以反演所需的量，即 $(m_1, k_4)$。

### 注意

现在我们将介绍 [`spring_mass_inverse.py`](../../source_code/spring_mass/spring_mass_inverse.py) 脚本的内容。

## 定义逆问题的方程、网络和节点

由于问题的一部分涉及记忆给定的解，我们将 `'t'` 作为输入 NumPy 数组，并将 `'x_1', 'x_2', 'x_3', 'ode_x1', 'ode_x2', 'ode_x3'` 作为输出 NumPy 数组。将 `'x_1', 'x_2', 'x_3'` 设置为来自解析数据的输入值，本质上是让网络吸收这些变量在选定域中的解析分布。将 `'ode_x1', 'ode_x2', 'ode_x3'` 设置为 0，我们还会通知网络在这些采样点处满足 ODE 损失函数。现在，假设我们也知道解的初始条件，除了 `'m_1'` 和 `'k_4'` 之外，这些微分方程中的所有变量都是已知的。因此，网络可以利用这些信息来反演未知数。以下代码展示了数据准备步骤。

```python
import torch
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import ModulusConfig, instantiate_arch
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Point1D
from modulus.geometry import Parameterization
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node

from spring_mass_ode import SpringMass

@modulus.main(config_path="conf", config_name="config_inverse")
def run(cfg: ModulusConfig) -> None:
    # prepare data
    t_max = 10.0
    deltaT = 0.01
    t = np.arange(0, t_max, deltaT)
    t = np.expand_dims(t, axis=-1)

    invar_numpy = {"t": t}
    outvar_numpy = {
        "x1": (1 / 6) * np.cos(t)
        + (1 / 2) * np.cos(np.sqrt(3) * t)
        + (1 / 3) * np.cos(2 * t),
        "x2": (2 / 6) * np.cos(t)
        + (0 / 2) * np.cos(np.sqrt(3) * t)
        - (1 / 3) * np.cos(2 * t),
        "x3": (1 / 6) * np.cos(t)
        - (1 / 2) * np.cos(np.sqrt(3) * t)
        + (1 / 3) * np.cos(2 * t),
    }
    outvar_numpy.update({"ode_x1": np.full_like(invar_numpy["t"], 0)})
    outvar_numpy.update({"ode_x2": np.full_like(invar_numpy["t"], 0)})
    outvar_numpy.update({"ode_x3": np.full_like(invar_numpy["t"], 0)})
```

为逆问题创建神经网络的过程与我们在前几节中看到的其他问题类似。然而，由于位移及其梯度（即速度和加速度）的信息已经存在（来自解析数据），可供网络记忆，因此我们将从各自的方程中分离这些变量和计算图。这意味着只有预测 `'m1'` 和 `'k4'` 的网络才会被优化，以最小化方程残差。位移、速度和加速度将被视为地面实况数据。

另外，请注意，质量和弹簧常数作为符号变量（分别为 `'m1'` 和 `'k4'` ）传入方程，因为它们在本问题中是未知数。

```python
    # make list of nodes to unroll graph on
    sm = SpringMass(k=(2, 1, 1, "k4"), m=("m1", 1, 1))
    sm_net = instantiate_arch(
        input_keys=[Key("t")],
        output_keys=[Key("x1"), Key("x2"), Key("x3")],
        cfg=cfg.arch.fully_connected,
    )
    invert_net = instantiate_arch(
        input_keys=[Key("t")],
        output_keys=[Key("m1"), Key("k4")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        sm.make_nodes(
            detach_names=[
                "x1",
                "x1__t",
                "x1__t__t",
                "x2",
                "x2__t",
                "x2__t__t",
                "x3",
                "x3__t",
                "x3__t__t",
            ]
        )
        + [sm_net.make_node(name="spring_mass_network", jit=cfg.jit)]
        + [invert_net.make_node(name="invert_network", jit=cfg.jit)]
    )
```

对于这个问题，我们将使用来自解析解的解，该解以我们之前定义的 NumPy 数组的形式存在。我们将使用 `PointwiseConstraint` 类来处理此类输入数据。`PointwiseConstraint` 类接收用于输入变量和输出变量的单独字典。这些字典为每个变量分配一个键，并创建一个与该键关联的 NumPy 数组，其中包含值。以下代码展示了如何添加边界约束以及来自我们刚刚定义的 NumPy 数组的约束。

```python
    # add constraints to solver
    # make geometry
    geo = Point1D(0)
    t_symbol = Symbol("t")
    x = Symbol("x")
    time_range = {t_symbol: (0, t_max)}

    # make domain
    domain = Domain()

    # initial conditions
    IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x1": 1.0, "x2": 0, "x3": 0, "x1__t": 0, "x2__t": 0, "x3__t": 0},
        batch_size=cfg.batch_size.IC,
        lambda_weighting={
            "x1": 1.0,
            "x2": 1.0,
            "x3": 1.0,
            "x1__t": 1.0,
            "x2__t": 1.0,
            "x3__t": 1.0,
        },
        parameterization=Parameterization({t_symbol: 0}),
    )
    domain.add_constraint(IC, name="IC")

    # data and pdes
    data = PointwiseConstraint.from_numpy(
        nodes=nodes,
        invar=invar_numpy,
        outvar=outvar_numpy,
        batch_size=cfg.batch_size.data,
    )
    domain.add_constraint(data, name="Data")
```

## 添加监视器

在本节中，我们将在求解过程中监视域内平均值 `'m_1'` 和 `'k_4'` 的收敛情况。一旦我们发现这些量的平均值达到稳定值，就可以结束模拟。生成此类监视器的代码如下所示。

```python
    # add monitors
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["m1"],
        metrics={"mean_m1": lambda var: torch.mean(var["m1"])},
        nodes=nodes,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["k4"],
        metrics={"mean_k4": lambda var: torch.mean(var["k4"])},
        nodes=nodes,
    )
    domain.add_monitor(monitor)
```

## 求解器和训练

现在我们已经完成了各种约束和域的定义，可以构建求解器并像之前一样运行问题。执行相同操作的代码如下所示。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

In [ ]:
%load_ext tensorboard
%tensorboard --host 0.0.0.0 --logdir outputs/spring_mass_inverse

In [ ]:
!python3 ../../source_code/spring_mass/spring_mass_inverse.py

## 可视化解决方案

我们可以监控 Tensorboard 绘图，查看模拟的收敛情况以及 $m_1$ 和 $k_4$ 的最终值。我们发现，经过充分训练后，$m_1$ 和 $k_4$ 分别为 1 和 2。这些值与我们创建解析解时使用的值相同。

## 下一步

在下一个笔记本中，我们将介绍傅里叶神经算子 (FNO) 及其一些变体背后的一些理论。之后，我们将以数据驱动的方式将这些架构应用于达西流场预测问题。

请继续阅读[下一个笔记本](../Operators/FNO_Darcy.ipynb)。